In [2]:
import numpy as np
import pandas as pd


train = pd.read_csv("/kaggle/input/big-data-analytics-certification/t2-1-train.csv")
test = pd.read_csv("/kaggle/input/big-data-analytics-certification/t2-1-test.csv")

In [ ]:
"""
[분류] 여행 보험 패키지 상품을 구매할 확률 값을 구하시오 / 리더보드 제출용
예측할 값(y): TravelInsurance (여행보험 패지지를 구매 했는지 여부 0:구매안함, 1:구매)
평가: roc-auc 평가지표
data: t2-1-train.csv, t2-1-test.csv


제출 형식
id,TravelInsurance
0,0.3
1,0.48
2,0.3
3,0.83
"""

In [3]:
train.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,10000,28,Private Sector/Self Employed,Yes,1250000.0,6,1,No,No,0
1,10001,31,Private Sector/Self Employed,Yes,1250000.0,7,1,No,No,0
2,10002,29,Private Sector/Self Employed,Yes,1200000.0,7,0,No,No,1
3,10003,33,Government Sector,Yes,650000.0,6,1,No,No,1
4,10004,28,Private Sector/Self Employed,Yes,800000.0,6,0,No,Yes,1


In [3]:
y_train, x_train = train.iloc[:,-1], train.iloc[:,:-1]

In [4]:
test.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,0,32,Government Sector,Yes,650000.0,5,0,No,No
1,1,27,Government Sector,Yes,500000.0,5,0,No,No
2,2,32,Government Sector,Yes,350000.0,2,0,No,No
3,3,26,Private Sector/Self Employed,Yes,1400000.0,6,0,No,Yes
4,4,27,Private Sector/Self Employed,No,350000.0,3,0,No,No


In [5]:
print(train.isnull().sum(),test.isnull().sum())

id                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           4
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64 id                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           3
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64


In [6]:
train[train["AnnualIncome"].isnull()] # Income 이 없는 것이 데이터의 특성인지 단순한 기입 오류인지 살펴봄, 직업이 존재하는데 income 이 없는 걸 보아 단순 기입 오류인듯.

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
12,10012,29,Private Sector/Self Employed,Yes,NaN,5,1,Yes,No,0
53,10053,34,Private Sector/Self Employed,Yes,NaN,3,0,No,No,0
214,10214,31,Government Sector,Yes,NaN,5,0,No,No,0
285,10285,35,Government Sector,Yes,NaN,8,0,Yes,No,1


In [7]:
m_1 = train[train["Employment Type"] == "Government Sector"]["AnnualIncome"].median()

In [8]:
m_2 = train[train["Employment Type"] == "Private Sector/Self Employed"]["AnnualIncome"].median()

In [9]:
train[(train["Employment Type"] == "Government Sector") & (train["AnnualIncome"].isnull())]["AnnualIncome"] = m_1
train[(train["Employment Type"] == "Private Sector/Self Employed") & (train["AnnualIncome"].isnull())]["AnnualIncome"] = m_2


/tmp/ipykernel_32/359295277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[(train["Employment Type"] == "Government Sector") & (train["AnnualIncome"].isnull())]["AnnualIncome"] = m_1
/tmp/ipykernel_32/359295277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[(train["Employment Type"] == "Private Sector/Self Employed") & (train["AnnualIncome"].isnull())]["AnnualIncome"] = m_2


In [10]:
id = test.pop('id')

In [11]:
test.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,32,Government Sector,Yes,650000.0,5,0,No,No
1,27,Government Sector,Yes,500000.0,5,0,No,No
2,32,Government Sector,Yes,350000.0,2,0,No,No
3,26,Private Sector/Self Employed,Yes,1400000.0,6,0,No,Yes
4,27,Private Sector/Self Employed,No,350000.0,3,0,No,No


In [12]:
x_test = test

In [13]:
y_train, x_train = train.iloc[:,-1], train.iloc[:,:-1]

In [14]:
x_train.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,10000,28,Private Sector/Self Employed,Yes,1250000.0,6,1,No,No
1,10001,31,Private Sector/Self Employed,Yes,1250000.0,7,1,No,No
2,10002,29,Private Sector/Self Employed,Yes,1200000.0,7,0,No,No
3,10003,33,Government Sector,Yes,650000.0,6,1,No,No
4,10004,28,Private Sector/Self Employed,Yes,800000.0,6,0,No,Yes


In [15]:
x_train = x_train.drop('id', axis = 1)  # x_train.drop(labels, axis {0: rows, 1:columns}, columns)

In [16]:
x_train.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,28,Private Sector/Self Employed,Yes,1250000.0,6,1,No,No
1,31,Private Sector/Self Employed,Yes,1250000.0,7,1,No,No
2,29,Private Sector/Self Employed,Yes,1200000.0,7,0,No,No
3,33,Government Sector,Yes,650000.0,6,1,No,No
4,28,Private Sector/Self Employed,Yes,800000.0,6,0,No,Yes


In [17]:
x_test.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,32,Government Sector,Yes,650000.0,5,0,No,No
1,27,Government Sector,Yes,500000.0,5,0,No,No
2,32,Government Sector,Yes,350000.0,2,0,No,No
3,26,Private Sector/Self Employed,Yes,1400000.0,6,0,No,Yes
4,27,Private Sector/Self Employed,No,350000.0,3,0,No,No


In [18]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

cols = ["Employment Type", "GraduateOrNot", "FrequentFlyer","EverTravelledAbroad"]

for col in cols:
    encoder = LabelEncoder()
    x_train[col] = encoder.fit_transform(x_train[col])
    x_test[col] = encoder.fit_transform(x_test[col])

# 좀 더 상세한 예측을 위해 scaler 로 변환. 
# scaler 을 해제하고 싶을 경우 역정규화
# inverse_scaled_data = scaler.inverse_transform(scaled_data)

scaler = StandardScaler() # scaler 은 변수를 2차원 이상으로 받음.
x_train["AnnualIncome"] = scaler.fit_transform(x_train[["AnnualIncome"]])
x_test["AnnualIncome"] = scaler.fit_transform(x_test[["AnnualIncome"]])

In [19]:
x_train.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,28,1,1,0.847262,6,1,0,0
1,31,1,1,0.847262,7,1,0,0
2,29,1,1,0.714411,7,0,0,0
3,33,0,1,-0.746953,6,1,0,0
4,28,1,1,-0.348399,6,0,0,1


In [20]:
x_test.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,32,1,1,-0.763688,5,0,0,0
1,27,1,1,-1.159555,5,0,0,0
2,32,1,1,-1.555423,2,0,0,0
3,26,2,1,1.215650,6,0,0,1
4,27,2,0,-1.555423,3,0,0,0


In [22]:
y_train

0       0
1       0
2       1
3       1
4       1
       ..
1485    0
1486    0
1487    1
1488    0
1489    0
Name: TravelInsurance, Length: 1490, dtype: int64

In [21]:
print(x_train.shape, x_test.shape, y_train.shape)

(1490, 8) (497, 8) (1490,)


In [22]:
# model 생성

import lightgbm as lgbm

model = lgbm.LGBMClassifier()
model.fit(x_train, y_train)

LGBMClassifier()

In [23]:
predicted_y = model.predict(x_test)

In [24]:
result = pd.DataFrame({'id': id, 'TravelInsurance': predicted_y})

In [25]:
result.head()

,id,TravelInsurance
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0


In [26]:
result.to_csv('001.csv',index = False)